In [2]:
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import manhattan,euclidean
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from hdimvis.visualise_layouts_and_metrics.plot import show_layout, show_generation_metrics
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from time import perf_counter
import numpy as np
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle

In [3]:

sizes = np.rint(np.linspace(100, 100000, 10))

####################
Fetching the "globe" dataset
####################
Dataset loaded
Dataset shape: (1000003, 3)
####################


In [ ]:
num_repeats =3
results= np.zeros((4,15, num_repeats)) # 5 for - dataset size, 96, squad, hybrid,

In [ ]:

for j,size in enumerate(sizes):
    dataset = DataFetcher.fetch_data('globe', size=size)
    results[0,j] = size
    embedding_PCA = PCA(n_components=2, whiten=False, copy=True).fit_transform(dataset.data).astype(np.float64)
    embedding_PCA *= 10/np.std(embedding_PCA)

    for i in num_repeats:
        algo96 = Chalmers96(dataset=dataset, initial_layout=None,  distance_fn=euclidean, spring_constant=0.7,
                             sample_set_size=10, neighbour_set_size=5)
        start_96 = perf_counter()
        layout_96 = LayoutCreation.create_layout(algo96, no_iters=100)
        results[1,j,i] = perf_counter() - start_96

        sned = SNeD(dataset=dataset)
        start_sned = perf_counter()
        layout_squad = LayoutCreation().create_layout(sned, no_iters=1000, optional_metric_collection=None)
        results[2,j,i] = perf_counter() - start_sned

        # start_tsne = perf_counter()
        # embedding_tsne = TSNE(n_components=2, perplexity=500).fit_transform(dataset.data)
        # results[3,j,i] = perf_counter() - start_tsne
        #
        # start_umap = perf_counter()
        # embedding_umap = umap.UMAP(n_neighbors=500).fit_transform(dataset.data)
        # results[4,j,i] = perf_counter() - start_umap


output_dir= (Path(PROJECT_ROOT).joinpath(
    Path(f"experiments/confirming_time_complexity/out/"))).resolve().absolute()

path_to_pickle = (Path(output_dir).joinpath(Path(f"results.pickle"))).resolve()
with open(path_to_pickle, 'wb') as pickle_out:
    pickle.dump(results, pickle_out)